In [ ]:
!pip install -q --upgrade bitsandbytes wandb faiss-gpu sentence_transformers datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/huggingface/trl.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
import torch
from torch.nn import functional as F
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
    BitsAndBytesConfig,
)
from trl import SFTTrainer
from peft import LoraConfig

# To capture output from the installation command
%pip install -q accelerate peft bitsandbytes transformers trl
# To enable inline plotting
%matplotlib inline

In [ ]:
# Check GPU status
!nvidia-smi

# Determine device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Mon Nov 11 16:43:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Question 1:
 Finetune the base language model (llama2 7B) with (output, instruction) pairs {(yi, xi)} from the seed data to obtain a backward model Myx := p(x|y). In other words, finetune a model that uses the output to predict the instruction. Use the openassistant-guanaco training set dataset. (25 points)

In [ ]:
from transformers import AutoTokenizer

# Loading tokenizer for the model
model_name = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set padding token
if not tokenizer.pad_token:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# Load and divide the dataset into training and test subsets
dataset = load_dataset('timdettmers/openassistant-guanaco')['train'].train_test_split(test_size=0.1)

# Apply a filter to retain only entries where tokenized text length is under 256 tokens
dataset = dataset.filter(lambda entry: len(tokenizer.tokenize(entry['text'])) < 256)

print(dataset)

# Prompt for crafting an instruction based on a given response
prompt_backward = """<s>[INST]Given below is a response. Create an instruction that fits this response.
### Response:
{}

### Instruction:
{}[/INST]"""

# Prompt for generating a response to a given instruction
prompt_forward = """<s>[INST]Given below is an instruction. Craft a response that answers it appropriately.
### Instruction:
{}

### Response:
{}[/INST]"""

EOS_TOKEN = tokenizer.eos_token

README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


openassistant_best_replies_train.jsonl:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

openassistant_best_replies_eval.jsonl:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8861 [00:00<?, ? examples/s]

Filter:   0%|          | 0/985 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3057
    })
    test: Dataset({
        features: ['text'],
        num_rows: 310
    })
})


In [ ]:
splits = ['train', 'test']
# Prepare lists for backward and forward prompts
backward_prompts = []
forward_prompts = []

for split in splits:
    for entry in dataset[split]["text"]:
        # Divide the entry text into relevant parts
        parts = [segment for segment in entry.split("### ") if len(segment) > 0][:2]

        # Continue only if the first part is labeled as 'Human' and the second as 'Assistant'
        if not (parts[0].startswith('Human: ') and parts[1].startswith('Assistant: ')):
            continue

        # Extract instruction and response, removing labels
        instruction_text = parts[0].replace("Human: ", "")
        response_text = parts[1].replace("Assistant: ", "")

        # Add backward and forward prompts to respective lists
        backward_prompts.append(prompt_backward.format(response_text, instruction_text) + EOS_TOKEN)
        forward_prompts.append(prompt_forward.format(instruction_text, response_text) + EOS_TOKEN)

from datasets import Dataset
dataset = Dataset.from_dict({"text": backward_prompts})
dataset_fwd = Dataset.from_dict({"text": forward_prompts})

In [ ]:
dataset["text"][0]

'<s>[INST]Given below is a response. Create an instruction that fits this response.\n### Response:\nEl libro «El Hombre más rico de Babilonia» es una obra de ficción escrita por George S. Clason que presenta enseñanzas financieras en formato de historias y parábolas. Uno de los principales temas del libro es la importancia de establecer metas financieras y hacer planes para alcanzarlas. El libro también destaca la importancia de buscar asesoramiento de expertos en finanzas antes de tomar decisiones importantes. La publicación enfatiza la importancia de tener un enfoque a largo plazo y evitar tomar decisiones financieras impulsivas. El autor también enseña la importancia de establecer un presupuesto, gastar menos de lo que se gana y ahorrar un porcentaje de cada ingreso.\n\n### Instruction:\nHazme un resumen de el libro “el hombre más rico de Babilonia”[/INST]</s>'

In [ ]:
dataset_fwd["text"][0]

'<s>[INST]Given below is an instruction. Craft a response that answers it appropriately.\n### Instruction:\nHazme un resumen de el libro “el hombre más rico de Babilonia”\n\n### Response:\nEl libro «El Hombre más rico de Babilonia» es una obra de ficción escrita por George S. Clason que presenta enseñanzas financieras en formato de historias y parábolas. Uno de los principales temas del libro es la importancia de establecer metas financieras y hacer planes para alcanzarlas. El libro también destaca la importancia de buscar asesoramiento de expertos en finanzas antes de tomar decisiones importantes. La publicación enfatiza la importancia de tener un enfoque a largo plazo y evitar tomar decisiones financieras impulsivas. El autor también enseña la importancia de establecer un presupuesto, gastar menos de lo que se gana y ahorrar un porcentaje de cada ingreso.[/INST]</s>'

#### Fine-tuning both forward and backward models

In [ ]:
max_length = 256

# Function to tokenize the prompt and generate labels
def tokenize_prompt(prompt):
    tokenized_output = tokenizer(
        prompt['text'],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    tokenized_output["labels"] = tokenized_output["input_ids"].copy()
    return tokenized_output

# Apply the tokenization function to the datasets
tokenized_dataset = dataset.map(tokenize_prompt)
tokenized_dataset_fwd = dataset_fwd.map(tokenize_prompt)

Map:   0%|          | 0/3367 [00:00<?, ? examples/s]

Map:   0%|          | 0/3367 [00:00<?, ? examples/s]

In [ ]:
# Define BitsAndBytes configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=getattr(torch, "float16"),
    bnb_4bit_use_double_quant=False,
)

# Load the model with the specified quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto"
)

# Set model configuration to disable caching and set pretraining tp
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

# Configure Lora for efficient fine-tuning of specific model layers
config = LoraConfig(
    r=32,  # Rank chosen for a balance between performance and memory
    lora_alpha=64,  # Higher alpha to capture more model complexity
    target_modules=[  # Applying Lora to key layers for attention and output optimization
        "q_proj",    # Query projection – crucial for attention patterns
        "k_proj",    # Key projection – impacts attention dynamics
        "v_proj",    # Value projection – completes attention mechanism
        "fc1",       # First MLP layer – tweaking non-linearity
        "fc2",       # Second MLP layer – deeper task optimization
        "dense",     # Dense layer – adding flexibility to the model
        "lm_head"    # LM head – adjusting output logits for better task performance
    ],
    bias="none",  # No bias terms in the Lora layers to simplify the model
    lora_dropout=0.05,  # Dropout to prevent overfitting while tuning
    task_type="CAUSAL_LM",  # For causal language modeling tasks
)

# Apply the Lora configuration to the model for parameter-efficient fine-tuning
model = get_peft_model(model, config)

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Hyperparameters for fine-tuning
learning_rate = 1e-5
weight_decay = 0.1
batch_size = 32

# Training arguments for the Trainer
training_args = TrainingArguments(
    output_dir="./q1",  # Directory to save model checkpoints
    per_device_train_batch_size=1,  # Batch size per device (adjust based on memory)
    gradient_accumulation_steps=1,  # Accumulate gradients before updating model
    lr_scheduler_type='cosine',  # Using cosine learning rate schedule
    max_steps=50,  # Limit number of training steps
    learning_rate=2e-5,  # Learning rate for training
    optim="paged_adamw_8bit",  # Optimizer with 8-bit precision for efficiency
    logging_steps=5,  # Log every 5 steps for tracking progress
)

# Initialize Trainer with model, dataset, and training arguments
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,  # Tokenized training dataset
    args=training_args,  # Training arguments
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)  # No MLM for causal LM task
)

# Start training the model
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
5,3.324100
10,3.295700
15,2.515000
20,2.949800
25,2.454700
30,2.401600
35,2.728000
40,2.337800
45,3.083700
50,2.333100


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.7423508644104, metrics={'train_runtime': 245.4965, 'train_samples_per_second': 0.204, 'train_steps_per_second': 0.204, 'total_flos': 509465434521600.0, 'train_loss': 2.7423508644104, 'epoch': 0.01485001485001485})

In [ ]:
# Initialize and train the forward model with the specified dataset and training settings
trainer_fwd = Trainer(
    model=model,
    train_dataset=tokenized_dataset_fwd,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer_fwd.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
5,2.756400
10,2.571500
15,1.988600
20,2.401300
25,2.003300
30,1.939600
35,2.111400
40,1.996000
45,2.452500
50,1.979000


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.21995662689209, metrics={'train_runtime': 53.4023, 'train_samples_per_second': 0.936, 'train_steps_per_second': 0.936, 'total_flos': 509465434521600.0, 'train_loss': 2.21995662689209, 'epoch': 0.01485001485001485})

#### Push model to hugging face

In [ ]:
from huggingface_hub import notebook_login

# Use the notebook_login function to log in
notebook_login()

In [ ]:
# Push the trained model to Hugging Face hub
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

events.out.tfevents.1731344722.40ec439632b6.1278.0:   0%|          | 0.00/8.03k [00:00<?, ?B/s]

events.out.tfevents.1731345034.40ec439632b6.1278.1:   0%|          | 0.00/8.03k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/skm04/q1/commit/520365f19a2540bcb70f0619a86a1e823ea92d01', commit_message='End of training', commit_description='', oid='520365f19a2540bcb70f0619a86a1e823ea92d01', pr_url=None, repo_url=RepoUrl('https://huggingface.co/skm04/q1', endpoint='https://huggingface.co', repo_type='model', repo_id='skm04/q1'), pr_revision=None, pr_num=None)

### Question 2:
Self-Augmentation -- Randomly sample a subset of size 150 and generate instructions from the LIMA dataset’s completions and filtering out any mutli-turn examples. Print out 5 examples of generated instructions. (25 points)

In [ ]:
# Load the 'lima' dataset from the GAIR repository
dataset = load_dataset('GAIR/lima')

In [ ]:
# Filter the dataset to keep only entries with fewer than 3 conversations
dataset = dataset.filter(lambda x: len(x['conversations']) < 3)
print(dataset)

Filter:   0%|          | 0/1030 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conversations', 'source'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['conversations', 'source'],
        num_rows: 300
    })
})


In [ ]:
generated_instructions = []  # List to store the generated instructions
max_conversations = 100  # Maximum number of conversations to process
counter = 0  # Initialize the counter

for conversation in dataset["train"]:
    # Stop if the maximum conversation limit is reached
    if counter >= max_conversations:
        break

    question, answer = conversation["conversations"][0], conversation["conversations"][1]

    # Generate a prompt for the backward model to create a relevant question based on the answer
    prompt = f"<s>[INST]Below is a user's answer. Your task is to create a question that fits this answer. Keep the question brief and direct, without any unnecessary elaboration. Format your response as: GENERATED INSTRUCTION:- [/INST]\n {answer}"

    # Tokenize the prompt
    tokenized_input = tokenizer.encode(prompt, return_tensors="pt")

    # Generate instruction using the fine-tuned backward model
    generated_ids = trainer.model.generate(tokenized_input, max_new_tokens=512)
    generated_question = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Append the result to the instructions list
    generated_instructions.append({"question": question, "answer": answer, "generated_question": generated_question})

    # Print progress every 10 steps
    if counter % 10 == 0:
        print(f"Completed {counter} predictions")

    counter += 1

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2113: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Completed 0 predictions
Completed 10 predictions
Completed 20 predictions
Completed 30 predictions
Completed 40 predictions
Completed 50 predictions
Completed 60 predictions
Completed 70 predictions
Completed 80 predictions
Completed 90 predictions


In [ ]:
# Refine and clean the generated instructions
refined_instructions = []  # List to store cleaned instructions
for entry in generated_instructions:
    question = entry["question"]
    answer = entry["answer"]
    generated_question = entry["generated_question"]

    # Extract the instruction from the generated question
    refined_instruction = generated_question.rsplit("GENERATED INSTRUCTION:")[-1].strip()

    refined_instructions.append({"original_question": question, "answer": answer, "refined_instruction": refined_instruction})

In [ ]:
refined_instructions

[{'original_question': 'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).',
  'answer': 'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).\nNeuronal s

# Question 3 :
Self curation (selecting high quality examples) using few shot prompting in addition to the prompt in Table 1 of the paper. Print out 5 examples of high quality examples and 5 examples of low quality examples. (25 points)


In [ ]:
# Prompt for evaluating the quality of a candidate answer based on user instruction
evaluation_prompt = """Below is a user's instruction and a candidate response.
Evaluate whether the answer effectively satisfies the user's instruction and assign it a score based on the following scale:
1: Answer is incomplete, vague, irrelevant, or off-topic. It may include extraneous content like personal opinions, promotional material, or unrelated information.
2: Answer addresses most of the user’s request but lacks detail or precision. It provides a general approach rather than a direct solution.
3: Answer is helpful but not AI-generated. It provides a solid answer but looks more like an excerpt from a blog or external source, rather than a tailored response from an AI assistant.
4: Answer is from an AI assistant’s perspective, clear, detailed, and directly answers the user’s query, with minor room for refinement (e.g., more concise).
5: Answer is a perfect AI Assistant response, fully addressing the user’s question with expert knowledge. The content is clear, well-organized, insightful, and free of irrelevant information.
First, provide a brief justification for the score you assign, and then conclude with "Score: <rating>".

Instruction: {}\n
Candidate Answer: {}
"""

max_conversations = 100  # Set maximum number of conversations to process
counter = 0  # Initialize conversation counter
evaluation_scores = []  # List to store evaluation results

for instruction in refined_instructions:
    # Stop processing once the max number of conversations is reached
    if counter >= max_conversations:
        break

    generated_instruction = instruction["refined_instruction"]
    answer = instruction["answer"]

    # Format the evaluation prompt with generated instruction and answer
    prompt = evaluation_prompt.format(generated_instruction, answer)

    # Tokenize the formatted prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate evaluation score using the fine-tuned forward model
    generated_ids = trainer_fwd.model.generate(input_ids, max_new_tokens=512)  # Adjust as needed
    score = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Store the generated instruction, answer, and the corresponding score
    evaluation_scores.append({"refined_instruction": generated_instruction, "answer": answer, "score": score})

    # Print progress every 10 steps
    if counter % 10 == 0:
        print(f"Completed {counter} evaluations")

    counter += 1

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2113: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Completed 0 evaluations
Completed 10 evaluations
Completed 20 evaluations
Completed 30 evaluations
Completed 40 evaluations
Completed 50 evaluations
Completed 60 evaluations
Completed 70 evaluations
Completed 80 evaluations
Completed 90 evaluations


In [ ]:
# Process the evaluation scores for each generated instruction
score_results = []
for score_entry in evaluation_scores:
    generated_instruction = score_entry["refined_instruction"]
    answer = score_entry["answer"]

    try:
        # Extract the score string from the response
        score_text = score_entry["score"].rsplit("Score:", 1)[-1].strip()

        # split the score text and get only the first valid number
        score_parts = score_text.split()
        score = score_parts[0].strip()

        # Ensure the score is a digit, otherwise handle it gracefully
        if score.isdigit():
            score = int(score)
        else:
            raise ValueError(f"Non-numeric score found in '{score_text}'.")

        # Append to the results
        score_results.append({"refined_instruction": generated_instruction, "answer": answer, "score": score})

    except Exception as e:
        # Handle any errors gracefully and log them for debugging
        print(f"Error processing score for {score_entry}: {str(e)}")
        score_results.append({"refined_instruction": generated_instruction, "answer": answer, "score": 0})

Error processing score for {'refined_instruction': '- [/INST]\n Sure, please take a look at csvkit. It provides a set of tools that adhere to the UNIX philosophy (meaning they are small, simple, single-purposed and can be combined). \n\nHere is an example that extracts the ten most populated cities in Germany from the free Maxmind World Cities database and displays the result in a console-readable format:\n```$ csvgrep -e iso-8859-1 -c 1 -m "de" worldcitiespop | csvgrep -c 5 -r "\\d+"\n  | csvsort -r -c 5 -l | csvcut -c 1,2,4,6 | head -n 11 | csvlook\n-----------------------------------------------------\n|  line_number | Country | AccentCity | Population  |\n-----------------------------------------------------\n|  1           | de      | Berlin     | 3398362     |\n|  2           | de      | Hamburg    | 1733846     |\n|  3           | de      | Munich     | 1246133     |\n|  4           | de      | Cologne    | 968823      |\n|  5           | de      | Frankfurt  | 648034      |\n| 

In [ ]:
score_results

[{'refined_instruction': '- [/INST]\n The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).\nNeuronal stem cells migrate over long distances in response to injury (Imitola et al., 2004) and they migrate fr

In [ ]:
# Filter instruction-answer pairs based on a minimum score
score_cutoff = 4
high_score_pairs = [entry for entry in score_results if entry["score"] >= score_cutoff]
high_score_pairs  # Display filtered results

[{'refined_instruction': "- [/INST]\n I'll answer this question from the theoretical side. The exponential behavior follows simply from the Schrödinger equation. Consider the one-electron Schrödinger equation:\n$$\n(-\\frac{1}{2}\\nabla^2 + V(\\mathbf{r}))\\psi(\\mathbf{r}) = \\epsilon\\psi(\\mathbf{r}), \\epsilon < 0\n$$\nAt spatial points that are very far away from the nucleus, $V(\\mathbf{r})\\approx 0$, so that the asymptotic solution is given by\n$$\n-\\frac{1}{2}\\nabla^2\\psi(\\mathbf{r}) = \\epsilon\\psi(\\mathbf{r}), \\epsilon < 0\n$$\nThis differential equation has basic solutions of the form\n$$\n\\psi(\\mathbf{r}) = Ce^{-\\sqrt{-2\\epsilon}\\mathbf{k}\\cdot\\mathbf{r}}\n$$\nfor some unit vector $\\mathbf{k}$. The real asymptotic behavior of $\\psi(\\mathbf{r})$ is thus a linear combination of these basic solutions. The linear combination may bring a polynomial prefactor to the exponential, but will never alter the exponent. Thus we have not only proved the exponential beha

In [ ]:
# Display the count before and after filtering, with the threshold applied
print(f"Total Samples Before Filtering: {len(score_results)}")
print(f"Total Samples After Applying Threshold: {len(high_score_pairs)}")
print(f"Threshold Used for Filtering: >= {score_cutoff}")

Total Samples Before Filtering: 100
Total Samples After Applying Threshold: 7
Threshold Used for Filtering: >= 4


In [ ]:
# Template for generating forward model prompts
prompt_format = """<s>[INST]The following is an instruction. Provide an appropriate response to address the question.
### Instruction:
{}

### Response:
{}[/INST]"""

# Define the end-of-sequence token
end_of_sequence_token = tokenizer.eos_token

# List to store the generated prompt texts
generated_prompts = []
for data in high_score_pairs:
    # Format each prompt using the instruction and the corresponding answer
    formatted_prompt = prompt_format.format(data["refined_instruction"], data["answer"])
    generated_prompts.append({"text": formatted_prompt})

generated_prompts

[{'text': "<s>[INST]The following is an instruction. Provide an appropriate response to address the question.\n### Instruction:\n- [/INST]\n I'll answer this question from the theoretical side. The exponential behavior follows simply from the Schrödinger equation. Consider the one-electron Schrödinger equation:\n$$\n(-\\frac{1}{2}\\nabla^2 + V(\\mathbf{r}))\\psi(\\mathbf{r}) = \\epsilon\\psi(\\mathbf{r}), \\epsilon < 0\n$$\nAt spatial points that are very far away from the nucleus, $V(\\mathbf{r})\\approx 0$, so that the asymptotic solution is given by\n$$\n-\\frac{1}{2}\\nabla^2\\psi(\\mathbf{r}) = \\epsilon\\psi(\\mathbf{r}), \\epsilon < 0\n$$\nThis differential equation has basic solutions of the form\n$$\n\\psi(\\mathbf{r}) = Ce^{-\\sqrt{-2\\epsilon}\\mathbf{k}\\cdot\\mathbf{r}}\n$$\nfor some unit vector $\\mathbf{k}$. The real asymptotic behavior of $\\psi(\\mathbf{r})$ is thus a linear combination of these basic solutions. The linear combination may bring a polynomial prefactor t

#### Push dataset to hugging face

In [ ]:
from huggingface_hub import notebook_login

# Log in to Hugging Face Hub using the notebook login function
notebook_login()

# Create a new dataset from the list of generated prompts
prompts_dataset = Dataset.from_list(generated_prompts)

# Upload the dataset to the Hugging Face Hub with a new name
prompts_dataset.push_to_hub("skm04/prompts_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/skm04/prompts_dataset/commit/aac3d4879be7f2cd27fa26f75d4a6b1fefeeb43a', commit_message='Upload dataset', commit_description='', oid='aac3d4879be7f2cd27fa26f75d4a6b1fefeeb43a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/skm04/prompts_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='skm04/prompts_dataset'), pr_revision=None, pr_num=None)

# Question 4 :
Finetune base model on dataset generated by step 3. Print out 5 example responses. (25 points)

In [ ]:
from datasets import load_dataset

# Load the dataset from the Hugging Face Hub
dataset_from_hub = load_dataset("skm04/prompts_dataset")

# Split the dataset into training and testing sets, using 10% for testing
split_data = dataset_from_hub['train'].train_test_split(test_size=0.1)

# Print the resulting dataset splits
print(split_data)

README.md:   0%|          | 0.00/265 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/26.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})


In [ ]:
# Define the available splits
splits = ['train']

# Initialize list for processed text
processed_texts = []

# Iterate through the available splits and append text data with EOS token
for split in splits:
    for text in dataset_from_hub[split]["text"]:
        processed_texts.append(text + EOS_TOKEN)

# Convert the processed texts into a Dataset format
final_dataset = Dataset.from_dict({"text": processed_texts})

print(final_dataset)

Dataset({
    features: ['text'],
    num_rows: 7
})


In [ ]:
final_dataset['text'][0]

"<s>[INST]The following is an instruction. Provide an appropriate response to address the question.\n### Instruction:\n- [/INST]\n I'll answer this question from the theoretical side. The exponential behavior follows simply from the Schrödinger equation. Consider the one-electron Schrödinger equation:\n$$\n(-\\frac{1}{2}\\nabla^2 + V(\\mathbf{r}))\\psi(\\mathbf{r}) = \\epsilon\\psi(\\mathbf{r}), \\epsilon < 0\n$$\nAt spatial points that are very far away from the nucleus, $V(\\mathbf{r})\\approx 0$, so that the asymptotic solution is given by\n$$\n-\\frac{1}{2}\\nabla^2\\psi(\\mathbf{r}) = \\epsilon\\psi(\\mathbf{r}), \\epsilon < 0\n$$\nThis differential equation has basic solutions of the form\n$$\n\\psi(\\mathbf{r}) = Ce^{-\\sqrt{-2\\epsilon}\\mathbf{k}\\cdot\\mathbf{r}}\n$$\nfor some unit vector $\\mathbf{k}$. The real asymptotic behavior of $\\psi(\\mathbf{r})$ is thus a linear combination of these basic solutions. The linear combination may bring a polynomial prefactor to the expo

In [ ]:
# Step 1: Define the maximum token length for truncation
max_token_length = 256  # Set the maximum length for tokenization

# Step 2: Function to tokenize the input text and prepare labels
def tokenize_and_prepare_prompt(data_point):
    tokenized = tokenizer(
        data_point['text'],  # Tokenizing the input text
        truncation=True,
        max_length=max_token_length,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # Copying input_ids to labels
    return tokenized

# Step 3: Apply the tokenization function to the dataset
tokenized_dataset = final_dataset.map(tokenize_and_prepare_prompt)

# Step 4: Set up the configuration for 4-bit quantization
bnb_quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Set the quantization type
    bnb_4bit_compute_dtype=getattr(torch, "float16"),  # Set the data type for computation
    bnb_4bit_use_double_quant=False,  # Disable double quantization
)

# Step 5: Load the model with quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_quantization_config,
    device_map="auto"  # Automatically assign devices for model parallelism
)

# Step 6: Configure model settings
model.config.use_cache = False  # Disable caching during generation
model.config.pretraining_tp = 1  # Set tensor parallelism for pretraining

# Step 7: Define and apply the LORA configuration for model fine-tuning
from peft import LoraConfig, get_peft_model

lora_configuration = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "fc1", "fc2", "dense", "lm_head"
    ],
    bias="none",
    lora_dropout=0.05,  # Apply dropout to the LORA layers
    task_type="CAUSAL_LM",  # Fine-tuning for causal language modeling
)

# Step 8: Apply LORA fine-tuning to the model
model = get_peft_model(model, lora_configuration)

# Step 9: Set up the training arguments for the model
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_config = TrainingArguments(
    output_dir="./q4",  # Directory to store the trained model
    per_device_train_batch_size=1,  # Batch size for training on each device
    gradient_accumulation_steps=1,  # No gradient accumulation (for smaller batches)
    lr_scheduler_type='cosine',  # Use a cosine learning rate scheduler
    max_steps=50,  # Set the total number of training steps
    learning_rate=2e-5,  # Small learning rate for fine-tuning
    optim="paged_adamw_8bit",  # Use the 8-bit Adam optimizer with paging
    logging_steps=5,  # Log training progress every 5 steps
)

# Step 10: Initialize the Trainer with the model, dataset, and training configuration
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_config,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)  # Disable MLM (Masked LM)
)

# Step 11: Start training the model
trainer.train()

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
5,3.186100
10,2.709000
15,2.887700
20,2.628200
25,2.840200
30,2.563800
35,2.610100
40,2.524700
45,2.803000
50,2.385700


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.7138596534729005, metrics={'train_runtime': 46.7557, 'train_samples_per_second': 1.069, 'train_steps_per_second': 1.069, 'total_flos': 509465434521600.0, 'train_loss': 2.7138596534729005, 'epoch': 7.142857142857143})

In [ ]:
from huggingface_hub import notebook_login

# Authenticate with the Hugging Face Hub using the notebook login function
notebook_login()

# Upload the trained model to your Hugging Face account for easy access
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

events.out.tfevents.1731350663.40ec439632b6.1278.3:   0%|          | 0.00/8.03k [00:00<?, ?B/s]

events.out.tfevents.1731350330.40ec439632b6.1278.2:   0%|          | 0.00/8.03k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/skm04/q4/commit/c57aaef7183160f665064e07bbc27f270d58c9c4', commit_message='End of training', commit_description='', oid='c57aaef7183160f665064e07bbc27f270d58c9c4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/skm04/q4', endpoint='https://huggingface.co', repo_type='model', repo_id='skm04/q4'), pr_revision=None, pr_num=None)

In [ ]:
print('URL: https://huggingface.co/skm04/q4/commit/c57aaef7183160f665064e07bbc27f270d58c9c4')

URL: https://huggingface.co/skm04/q4/commit/c57aaef7183160f665064e07bbc27f270d58c9c4
